# Watersheds Segmentation <a href="https://mybinder.org/v2/gh/InsightSoftwareConsortium/SimpleITK-Notebooks/master?filepath=Python%2F32_Watersheds_Segmentation.ipynb"><img style="float: right;" src="https://mybinder.org/badge_logo.svg"></a>

In [47]:
%matplotlib inline
import matplotlib.pyplot as plt
import SimpleITK as sitk
from myshow import myshow, myshow3d
import numpy as np

# Download data to work on
%run update_path_to_download_script
from downloaddata import fetch_data as fdata

import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import random

<Figure size 432x288 with 0 Axes>

In [48]:
from os import walk

f = []
for (dirpath, dirnames, filenames) in walk('./data'):
    f.extend(dirnames)
    break
print(f)

['604.000000-COR 3X3-11320', '3.000000-0.625mm bone alg-26970', '2.000000-ROUTINE CHEST NON-CON-97100', '602.000000-SAG 3X3-00291', '605.000000-SAG 3X3-10651', '601.000000-COR 3X3-86740', '1.000000-SCOUT CHEST-97846']


In [49]:
reader = sitk.ImageSeriesReader()

def get_segementation_image(name):
    dicom_names = reader.GetGDCMSeriesFileNames(name)
    reader.SetFileNames(dicom_names)
    image = reader.Execute()
    segmentation(image)
#     myshow(image) 
    return sitk.GetArrayFromImage(image)

def segmentation(image):
    image = sitk.GradientMagnitude(image)
    return sitk.MorphologicalWatershed(image, level=20, markWatershedLine=True, fullyConnected=False)

def get_training_data(arr):
    print(arr)
    training_data = []
    first = True
    for fn in arr:
        path = str('./data/' + fn)
        print(path)
        image = get_segementation_image(path)
        print(len(image))
        if len(image) > 50:
            if first:
                training_data = [random.sample(list(image), 50)]
                first = False
            else:
                training_data = np.concatenate((training_data, [random.sample(list(image), 50)]))
        
    return training_data

training_data = get_training_data(f)




['604.000000-COR 3X3-11320', '3.000000-0.625mm bone alg-26970', '2.000000-ROUTINE CHEST NON-CON-97100', '602.000000-SAG 3X3-00291', '605.000000-SAG 3X3-10651', '601.000000-COR 3X3-86740', '1.000000-SCOUT CHEST-97846']
./data/604.000000-COR 3X3-11320


ImageSeriesReader (0x10cc06050): Non uniform sampling or missing slices detected,  maximum nonuniformity:176.649



111
./data/3.000000-0.625mm bone alg-26970
510
./data/2.000000-ROUTINE CHEST NON-CON-97100
255
./data/602.000000-SAG 3X3-00291


ImageSeriesReader (0x12e6c9620): Non uniform sampling or missing slices detected,  maximum nonuniformity:176.686



121
./data/605.000000-SAG 3X3-10651


ImageSeriesReader (0x12e6c9620): Non uniform sampling or missing slices detected,  maximum nonuniformity:176.682



121
./data/601.000000-COR 3X3-86740


ImageSeriesReader (0x12e6c9620): Non uniform sampling or missing slices detected,  maximum nonuniformity:176.653



106
./data/1.000000-SCOUT CHEST-97846
2


In [ ]:

hl = "./healthy_lung"
healthy_lung = get_segementation_image(hl)
training_data = np.concatenate((training_data, [random.sample(list(healthy_lung), 50)]))
print(np.shape(training_data))
print(training_data)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(50, 512, 512)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64))
model.add(layers.Dense(32))
model.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=3)

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['binary_accuracy'])


In [ ]:
labels = np.array([1] * (len(training_data) - 1))
labels = np.concatenate((labels, [0]))
print(labels)
history = model.fit(training_data/255, labels, epochs=10, validation_data=(training_data/255, labels), callbacks=callback)

In [34]:

# list
# print(data['labelGroups'][2])
for i in range(0, len(data['datasets'])):
    for j in range(0, len(data['datasets'][i]['annotations'])):
        if '1.2.826.0.1.3680043.10.474.419639.312580455409613733097488204614' == data['datasets'][i]['annotations'][j].get('StudyInstanceUID', 0):
            print(data['datasets'][i]['annotations'][j]['labelId'])
    

L_ngve9m
L_ngve9m
L_MAYK1M
L_MAYK1M
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_MAYK1M
L_MAYK1M
L_ngve9m
L_MAYK1M
L_MAYK1M
L_MAYK1M
L_ngve9m
L_ngve9m
L_ngve9m
L_mvd9bJ
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L_ngve9m
L

In [4]:
import json
# Opening JSON file
f = open('MIDRC-RICORD-1a_annotations_labelgroup_all_2020-Dec-8 2.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)
 
# Iterating through the json
# list
for i in data:
    print(i)
 
# Closing file
f.close()

id
createdAt
updatedAt
name
description
isPrivate
labelGroups
datasets


In [24]:
data['datasets'][0]['annotations'][0]['SeriesInstanceUID']

'1.2.826.0.1.3680043.10.474.3116300553210485620889426707485524974'

In [46]:
dicom_names = reader.GetGDCMSeriesFileNames('./data/2.000000-ROUTINE CHEST NON-CON-97100/')
reader.SetFileNames(dicom_names)
imagess = reader.Execute()
myshow(imagess)

interactive(children=(IntSlider(value=127, description='z', max=254), Output()), _dom_classes=('widget-interac…

In [34]:
feature_img = sitk.GradientMagnitude(imagess)
myshow(feature_img)

interactive(children=(IntSlider(value=127, description='z', max=254), Output()), _dom_classes=('widget-interac…

In [35]:
ws_img = sitk.MorphologicalWatershed(feature_img, level=30, markWatershedLine=True, fullyConnected=False)
myshow(sitk.LabelToRGB(ws_img), "Watershed Over Segmentation")

interactive(children=(IntSlider(value=127, description='z', max=254), Output()), _dom_classes=('widget-interac…

In [36]:
ws_img = sitk.MorphologicalWatershed(feature_img, level=50, markWatershedLine=True, fullyConnected=False)
myshow(sitk.LabelToRGB(ws_img), "Watershed Over Segmentation")

interactive(children=(IntSlider(value=127, description='z', max=254), Output()), _dom_classes=('widget-interac…

In [37]:
ws_img = sitk.MorphologicalWatershed(feature_img, level=20, markWatershedLine=True, fullyConnected=False)
myshow(sitk.LabelToRGB(ws_img), "Watershed Over Segmentation")

interactive(children=(IntSlider(value=127, description='z', max=254), Output()), _dom_classes=('widget-interac…

In [38]:
ws_img = sitk.MorphologicalWatershed(feature_img, level=10, markWatershedLine=True, fullyConnected=False)
myshow(sitk.LabelToRGB(ws_img), "Watershed Over Segmentation")

interactive(children=(IntSlider(value=127, description='z', max=254), Output()), _dom_classes=('widget-interac…